# ML Model Example

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [1693]:
import influxdb_client_3 as InfluxDBClient3
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
pio.renderers.default = "vscode"

Next we provide our connection credentials to the InfluxDB 3.0 instance. 
`token` - scoped to the bucket you wish to query from.
`host` - the host of your cloud dedicated instance.
`client` - host wrapped in the flightclient class. 

In [1694]:
token = ''
host = 'eu-central-1-1.aws.cloud2.influxdata.com'    
org = 'Jay-IOx'
db = 'factory'

client = InfluxDBClient3.InfluxDBClient3(token=token,
                         host=host,
                         org=org,
                         database=db) 
    
    


In [1695]:
query = "SELECT vibration FROM machine_data WHERE \"machineID\" = 'machine3' AND time >= now() - 5m"
table = client.query(query=query, language="influxql")
print(table)

pyarrow.Table
iox::measurement: string not null
time: timestamp[ns] not null
vibration: double
----
iox::measurement: [["machine_data","machine_data","machine_data","machine_data","machine_data",...,"machine_data","machine_data","machine_data","machine_data","machine_data"]]
time: [[2023-06-20 16:25:53.193953126,2023-06-20 16:25:54.195300296,2023-06-20 16:25:55.195596220,2023-06-20 16:25:56.196376559,2023-06-20 16:25:57.197194293,...,2023-06-20 16:30:38.412000575,2023-06-20 16:30:39.413244330,2023-06-20 16:30:40.413552965,2023-06-20 16:30:41.414622984,2023-06-20 16:30:42.415388298]]
vibration: [[81,81,87,89,85,...,81,88,86,80,82]]


The `query_tools` function provides a wrapper to execution of a query agaisnt InfluxDB usinf flight. This function requires 3 paramters.
`database`- this specifies the database you wish to direct your query at.
`query` - This accepts a string literal of the query you wish to perform (SQL or InfluxQL).
`query_type` - Whether this is a SQL or InfluxQL based query.

Return all columns from the table flight for the past 3 days.

In [1696]:
df = table.to_pandas()
display(df)

,iox::measurement,time,vibration
0,machine_data,2023-06-20 16:25:53.193953126,81.0
1,machine_data,2023-06-20 16:25:54.195300296,81.0
2,machine_data,2023-06-20 16:25:55.195596220,87.0
3,machine_data,2023-06-20 16:25:56.196376559,89.0
4,machine_data,2023-06-20 16:25:57.197194293,85.0
...,...,...,...
285,machine_data,2023-06-20 16:30:38.412000575,81.0
286,machine_data,2023-06-20 16:30:39.413244330,88.0
287,machine_data,2023-06-20 16:30:40.413552965,86.0
288,machine_data,2023-06-20 16:30:41.414622984,80.0


In [1697]:
fig = px.line(df, x="time", y=["vibration"], title='raw vibration')
fig.show()

In [1698]:
# Load the MLP model from the file
model = load_model('MLP.h5')

# Assuming your dataframe is loaded and named as df
# df = pd.read_csv('your_data.csv')

# Setting the index to be the time column and sorting
df = df.set_index('time').sort_index()

# Dropping unnecessary columns and keeping relevant features
df_fit = df.drop(columns=['iox::measurement'])

# Normalize your data
# Note: It's important to use the same scaler that was used during training
# You may need to save and load the scaler separately
scaler = StandardScaler()
anomalous_data = scaler.fit_transform(df_fit)

# Use the MLP to reconstruct the input data
reconstructed_data = model.predict(anomalous_data)

# Calculate the reconstruction error (MSE)
mse = np.mean(np.power(anomalous_data - reconstructed_data, 2), axis=1)

# Set the threshold for detecting anomalies
# Note: You may want to use the same threshold value that was determined during training
threshold = 0.50 # (set your threshold)

# Create a new column in the original DataFrame for anomaly tags
df['anomalies'] = mse > threshold

# Display the DataFrame with the new column indicating anomalies
print(df)

10/10 [==============================] - 0s 889us/step
                              iox::measurement  vibration  anomalies
time                                                                
2023-06-20 16:25:53.193953126     machine_data       81.0      False
2023-06-20 16:25:54.195300296     machine_data       81.0      False
2023-06-20 16:25:55.195596220     machine_data       87.0      False
2023-06-20 16:25:56.196376559     machine_data       89.0      False
2023-06-20 16:25:57.197194293     machine_data       85.0      False
...                                        ...        ...        ...
2023-06-20 16:30:38.412000575     machine_data       81.0      False
2023-06-20 16:30:39.413244330     machine_data       88.0      False
2023-06-20 16:30:40.413552965     machine_data       86.0      False
2023-06-20 16:30:41.414622984     machine_data       80.0      False
2023-06-20 16:30:42.415388298     machine_data       82.0      False

[290 rows x 3 columns]


In [1699]:
# Create a new figure
fig = go.Figure()


# Add a line for the actual load
fig.add_trace(go.Scatter(
    x=df.index,
    y=df["vibration"],
    mode='lines',
    name='vibration',
    line=dict(color='green'),  # specify line color here
))

# Add a line for the anomalous load
fig.add_trace(go.Scatter(
    x=df[df['anomalies']].index,
    y=df[df['anomalies']]["vibration"],
    mode='markers',
    name='Anomalous Vibration',
    marker=dict(color='red', size=5),  # specify marker color and size here
))

# Show the plot
fig.show()